In [3]:
import requests
import pandas as pd

In [4]:
'''
load_
transform_
export__to_gcp_bucket
'''

api_key = ""

resorts = [
    ("Ikon", "Winter Park", (39.87652886804281, -105.76577005628859)),
    ("Ikon", "Copper Mountain", (39.49367530426312, -106.15941614917506)),
    ("Ikon", "Eldora", (39.93747528686545, -105.58400897925249)),
    ("Ikon", "Steamboat Springs", (40.466202977197106, -106.78094073252413)),
    ("Ikon", "Palisades Tahoe", (39.19114916529031, -120.2557470349228)),
    ("Ikon", "Big Sky", (45.293131560098956, -111.36294562989895)),
    ("Epic", "Northstar", (39.25671866393303, -120.13341133625593)),
    ("Ikon", "Crystal", (46.934754314303206, -121.48384975195034)),
    ("Ikon", "Jackson", (43.594634283058724, -110.84610943388475)),
    ("Epic", "Breckenridge", (39.478861079038985, -106.07839392176015)),
    ("Epic", "Vail", (39.619779887440146, -106.36957399501661)),
    ("Ikon", "Stratton", (43.110007073773104, -72.91026718465827)),
    ("Epic", "Whistler", (50.073500220248334, -122.95937601933981)),
    ("Ikon", "Charmonix", (45.96980068114292, 6.879032518531963))
]

cols_forecast = [ 
       "pass",
       "resort",
       "date",
       "vis",
       "temp",
       "feels_like",
       "temp_min",
       "temp_max",
       "weather",
       "weather_description",
       "cloud_percent",
       "wind_speed",
       "latitude",
       "longitude",
       "rain_mm",
       "snow_mm"
]

cols_current=[ 
       "pass",
       "resort",
       "date",
       "vis",
       "temp",
       "feels_like",
       "temp_min",
       "temp_max",
       "weather",
       "weather_description",
       "cloud_percent",
       "wind_speed",
       "latitude",
       "longitude",
       "rain_3h_mm",
       "rain_1h_mm",
       "snow_1h_mm",
       "snow_3h_mm"
]

In [5]:
# Kelvin to Celsius
def kelvin_to_celsius(k):
    return k - 273.15

# Celsius to Fahrenheit
def celsius_to_fahrenheit(c):
    return c * 9/5 + 32

def kelvin_to_fahrenheit(k):
    return celsius_to_fahrenheit(kelvin_to_celsius(k))

# Millimeters to Inches
def mm_to_inches(mm):
    return mm * 0.0393701

# Meters per second to Miles per hour
def mps_to_mph(mps):
    return mps * 2.23694

In [6]:
def extract_open_weather_map_data(api_key, resorts, cols, weather_type):
    '''
    api_key: openweathermap.org api key
    cities: list of structure: [resort_name, (latitude, long)]
    cols: dataframe column names
    weather_type = "forecast" or "current"
    '''

    weather_data = {col: [] for col in cols}

    for pass_name, name, (lat, lon) in resorts:
        
        
        if weather_type == "forecast":
            url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}"
            process_func = process_forecast
        elif weather_type == "current":
            url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
            process_func = process_current
        else:
            return "Invalid Weather Type. Options: [forecast, current]"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
        
        else:
            print(f'Error fetching weather data: \nresponse: {response}\nurl: {url}')
            continue
        
        process_func(data, weather_data, (pass_name, name, lat, lon))
    return pd.DataFrame(weather_data)


In [7]:
def process_current(x, weather_data, resort_info):
    
    
    pass_name, name, lat, lon = resort_info
  
        
    # date
    date = x['dt']

    # visibility
    visibility = x['visibility']

    # temperature
    temp = x['main']['temp']
    feels_like = x['main']['feels_like']
    temp_min = x['main']['temp_min']
    temp_max = x['main']['temp_max']

    # snowfall / rainfall (previous 3 hours)
    if "rain" in x:
        if '1h' in x['rain']:
            rain_1h_mm = x['rain']['1h']
        else:
            rain_1h_mm = 0
        if '3h' in x['rain']:
            rain_3h_mm = x['rain']['3h']
        else:
            rain_3h_mm = 0
    else:
        rain_1h_mm, rain_3h_mm = 0,0
            

    if "snow" in x:
        if '1h' in x['snow']:
            snow_1h_mm = x['snow']['1h']
        else:
            snow_1h_mm = 0
        if '3h' in x['snow']: 
            snow_3h_mm = x['snow']['3h']
        else:
            snow_3h_mm = 0
    else:
        snow_1h_mm, snow_3h_mm = 0,0

    # weather
    weather = x['weather'][0]['main']
    desc = x['weather'][0]['description']

    cloud_percent = x['clouds']['all']
    wind_speed = x['wind']['speed']


    weather_data['pass'].append(pass_name)
    weather_data['resort'].append(name)
    weather_data['latitude'].append(lat)
    weather_data['longitude'].append(lon)
    weather_data['date'].append(date)
    weather_data['vis'].append(visibility)
    weather_data['temp'].append(temp)
    weather_data['feels_like'].append(feels_like) 
    weather_data['temp_min'].append(temp_min) 
    weather_data['temp_max'].append(temp_max) 
    weather_data['weather'].append(weather) 
    weather_data['weather_description'].append(desc) 
    weather_data['cloud_percent'].append(cloud_percent) 
    weather_data['wind_speed'].append(wind_speed) 
    
    weather_data['snow_1h_mm'].append(snow_1h_mm)
    weather_data['rain_1h_mm'].append(rain_1h_mm)
    weather_data['snow_3h_mm'].append(snow_3h_mm)
    weather_data['rain_3h_mm'].append(rain_3h_mm)
    
    
    return weather_data
    
    

In [8]:
'''
{'dt': 1707264000, 
 'main': {'temp': 276.34, 'feels_like': 275.18, 'temp_min': 272.67, 'temp_max': 276.34, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 702, 'humidity': 58, 'temp_kf': 3.67}, 
 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 
 'clouds': {'all': 29}, 'wind': {'speed': 1.4, 'deg': 155, 'gust': 1.95}, 
 'visibility': 10000, 
 'pop': 0, 
 'sys': {'pod': 'd'}, 
 'dt_txt': '2024-02-07 00:00:00'}
'''

"\n{'dt': 1707264000, \n 'main': {'temp': 276.34, 'feels_like': 275.18, 'temp_min': 272.67, 'temp_max': 276.34, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 702, 'humidity': 58, 'temp_kf': 3.67}, \n 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], \n 'clouds': {'all': 29}, 'wind': {'speed': 1.4, 'deg': 155, 'gust': 1.95}, \n 'visibility': 10000, \n 'pop': 0, \n 'sys': {'pod': 'd'}, \n 'dt_txt': '2024-02-07 00:00:00'}\n"

In [9]:
# process the forecast for a single resort forecast
def process_forecast(response_data, weather_data, resort_info):
    '''
    response_data: raw data from api (dict)
    weather_data: accumulator (dict)
    resort_info: (pass_name, name, latitude, longitude) (tuple)
    '''
    
    
    pass_name, name, lat, lon = resort_info
    
    for x in response_data['list']: # get first 8 values because only concerned with 24 hour forecast

        # date
        date = x['dt_txt']
        
        # visibility
        visibility = x['visibility']
        
        # temperature
        temp = x['main']['temp']
        feels_like = x['main']['feels_like']
        temp_min = x['main']['temp_min']
        temp_max = x['main']['temp_max']
        
        # snowfall / rainfall (previous 3 hours)
        if "rain" in x:
            rain_mm = x['rain']['3h']
        else:
            rain_mm = 0
        
        if "snow" in x:
            snow_mm = x['snow']['3h']
        else:
            snow_mm = 0
            
        
        # weather
        weather = x['weather'][0]['main']
        desc = x['weather'][0]['description']
        
        cloud_percent = x['clouds']['all']
        wind_speed = x['wind']['speed']
        
        prob_of_precipitation = x['pop']
        
        weather_data['pass'].append(pass_name)
        weather_data['resort'].append(name)
        weather_data['latitude'].append(lat)
        weather_data['longitude'].append(lon)
        weather_data['date'].append(date)
        weather_data['vis'].append(visibility)
        weather_data['temp'].append(temp)
        weather_data['feels_like'].append(feels_like) 
        weather_data['temp_min'].append(temp_min) 
        weather_data['temp_max'].append(temp_max) 
        weather_data['weather'].append(weather) 
        weather_data['weather_description'].append(desc) 
        weather_data['cloud_percent'].append(cloud_percent) 
        weather_data['wind_speed'].append(wind_speed) 
        weather_data['snow_mm'].append(snow_mm)
        weather_data['rain_mm'].append(rain_mm)
    
    return weather_data

In [10]:
df = extract_open_weather_map_data(api_key, resorts, cols_forecast, "forecast")
df

,pass,resort,date,vis,temp,feels_like,temp_min,temp_max,weather,weather_description,cloud_percent,wind_speed,latitude,longitude,rain_mm,snow_mm
0,Ikon,Winter Park,2024-03-02 00:00:00,10000,276.39,274.35,272.40,276.39,Clouds,scattered clouds,31,2.11,39.876529,-105.765770,0.0,0.0
1,Ikon,Winter Park,2024-03-02 03:00:00,10000,269.76,266.96,265.45,269.76,Clouds,few clouds,15,1.85,39.876529,-105.765770,0.0,0.0
2,Ikon,Winter Park,2024-03-02 06:00:00,10000,265.55,261.02,265.55,265.55,Clear,clear sky,4,2.58,39.876529,-105.765770,0.0,0.0
3,Ikon,Winter Park,2024-03-02 09:00:00,10000,265.63,260.97,265.63,265.63,Clear,clear sky,1,2.70,39.876529,-105.765770,0.0,0.0
4,Ikon,Winter Park,2024-03-02 12:00:00,10000,265.17,259.81,265.17,265.17,Clear,clear sky,1,3.23,39.876529,-105.765770,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,Ikon,Charmonix,2024-03-06 09:00:00,10000,265.86,265.86,265.86,265.86,Clouds,broken clouds,79,1.27,45.969801,6.879033,0.0,0.0
556,Ikon,Charmonix,2024-03-06 12:00:00,10000,268.03,268.03,268.03,268.03,Clouds,broken clouds,53,1.30,45.969801,6.879033,0.0,0.0
557,Ikon,Charmonix,2024-03-06 15:00:00,10000,268.90,268.90,268.90,268.90,Clouds,scattered clouds,29,0.90,45.969801,6.879033,0.0,0.0
558,Ikon,Charmonix,2024-03-06 18:00:00,10000,263.49,258.90,263.49,263.49,Clouds,few clouds,24,2.35,45.969801,6.879033,0.0,0.0


array(['2024-02-29 21:00:00', '2024-03-01 00:00:00',
       '2024-03-01 03:00:00', '2024-03-01 06:00:00',
       '2024-03-01 09:00:00', '2024-03-01 12:00:00',
       '2024-03-01 15:00:00', '2024-03-01 18:00:00'], dtype=object)

In [2]:
import datetime


now = datetime.datetime.now()
now

datetime.datetime(2024, 3, 5, 16, 20, 12, 241569)

In [3]:
f'{now.year}-{now.month}-{now.day}'

'2024-3-5'